In [ ]:
from web3 import Web3
import json
import web3
import os

In [ ]:
def read_json(path):
    with open(path, "r") as fin:
        return json.load(fin)

def get_contract_instance_from_bytecode(path_folder_build_contracts, name_contract):
    """
    Instantiate a non-deployed contract
    """
    path = f"{path_folder_build_contracts}{name_contract}"
    json_build = read_json(f"{path}.json")
    abi = json_build["abi"]
    bytecode = json_build["bytecode"]
    contract = w3.eth.contract(abi=abi, bytecode=bytecode)
    return contract

def get_contract_instance_from_address(path_folder_build_contracts, name_contract, address):
    """
    Instantiate an already deployed contract.
    """
    path = f"{path_folder_build_contracts}{name_contract}"
    json_build = read_json(f"{path}.json")
    abi = json_build["abi"]
    contract = w3.eth.contract(abi=abi, address=address)
    return contract

def get_balances(address, deployed_erc20_contract):
    """
    Get ether and ERC20 balances for address
    """
    balance_erc20_tokens = w3.fromWei(deployed_erc20_contract.functions.balanceOf(address).call(), unit="ether")
    balance_eth = w3.fromWei(w3.eth.get_balance(address), unit="ether")
    return balance_eth, balance_erc20_tokens

In [ ]:
class MVAMM:
    def __init__(self, dict_amm_contract, dict_erc20_contract):
        """
        Interface for interacting with deployed AMM.
        """
        self.amm_name = dict_amm_contract["name"]
        self.amm_address = dict_amm_contract["address"]
        self.json_folder = dict_amm_contract["path_json"]
        self.amm_contract = get_contract_instance_from_address(self.json_folder, self.amm_name, self.amm_address)
        
        self.erc20_name = dict_erc20_contract["name"]
        self.erc20_address = dict_erc20_contract["address"]
        self.json_folder = dict_erc20_contract["path_json"]
        self.erc20_contract = get_contract_instance_from_address(self.json_folder, self.erc20_name, self.erc20_address)
        
    def __approve_transaction(self, amount_t):
        """
        Private method for approving transactions involving ERC20 swappable token.
        """
        call_values = {"spender": self.amm_address, "amount": w3.toWei(amount_t, unit="ether")}
        tx = self.erc20_contract.functions.approve(**call_values).transact()
        return tx
    
    def get_LP_info(self):
        """
        Helper method to obtain AMM current liquidity state.
        """
        amountE = w3.fromWei(self.amm_contract.functions.balanceE().call(), unit="ether")
        amountT = w3.fromWei(self.amm_contract.functions.balanceT().call(), unit="ether")
        # Divide by 1e36 since both E and T are denominated in wei 1e18.
        amountL = w3.fromWei(self.amm_contract.functions.totalLiquidityTokenSupply().call(), unit="ether")
        k = self.amm_contract.functions.k().call()/1e36
        return amountE, amountT, amountL, k

    def get_balance_liquidity_tokens(self, address):
        """
        Getter method to obtain amount of liquidity tokens held by address.
        """
        balance_liquidity_tokens = w3.fromWei(self.amm_contract.functions.liquidityTokensHolders(address).call(), unit="ether")
        return balance_liquidity_tokens

    def initiate_liquidity(self, amount_liquidity_e, amount_liquidity_t):
        """
        Initiate liquidity for AMM. Provide amount E (ether) and T (erc20).
        """
        tx_approval = self.__approve_transaction(amount_liquidity_t)
        call_values = {"tokenAmount": w3.toWei(amount_liquidity_t, unit="ether")}
        transaction_values = {"value": w3.toWei(amount_liquidity_e, unit="ether")}
        tx = self.amm_contract.functions.initiateLiquidity(**call_values) \
                                    .transact(transaction_values)
        return tx

    def add_liquidity(self, amount_liquidity_e, amount_liquidity_t):
        """
        Add liquidity to the AMM.
        """
        tx_approval = self.__approve_transaction(amount_liquidity_t)
        call_values = {"tokenAmount": w3.toWei(amount_liquidity_t, unit="ether")}
        transaction_values = {"value": w3.toWei(amount_liquidity_e, unit="ether")}
        tx = self.amm_contract.functions.addLiquidity(**call_values) \
                                    .transact(transaction_values)
        return tx

    def remove_liquidity(self, amount_liquidity_tokens):
        """
        Remove liquidity from the AMM.
        """
        call_values = {"liquidityTokensAmount": w3.toWei(amount_liquidity_tokens, unit="ether")}
        tx = self.amm_contract.functions.removeLiquidity(**call_values).transact()
        return tx

    def get_input_price(self, amount):
        """
        Get input price (eth -> token)
        """
        call_values = {"amountE": w3.toWei(amount, unit="ether")} 
        price = self.amm_contract.functions.getInputPrice(**call_values).call()
        return w3.fromWei(price, unit="ether")

    def get_output_price(self, amount):
        """
        Get output price (token -> eth)
        """
        call_values = {"amountT": w3.toWei(amount, unit="ether")}
        price = self.amm_contract.functions.getOutputPrice(**call_values).call()
        return w3.fromWei(price, unit="ether")
    
    def eth_to_token(self, amount_e):
        """
        Swap an amount of ether for an amount of erc20 
        """
        transaction_values = {"value": w3.toWei(amount_e, unit="ether")}
        tx = self.amm_contract.functions.ethToToken().transact(transaction_values)
        return tx
    
    def token_to_eth(self, amount_t):
        """
        Swap an amount of erc20 for ether
        """
        tx_approval = self.__approve_transaction(amount_t)
        call_values = {"amountT": w3.toWei(amount_t, unit="ether")} 
        tx = self.amm_contract.functions.tokenToEth(**call_values).transact()
        return tx

In [ ]:
network = "http://127.0.0.1:7545"
path_to_build_folder = "../../solidity_projects/mvamm/build/contracts/"
w3 = Web3(Web3.HTTPProvider(network))
w3.eth.default_account = w3.eth.accounts[0]

### Deploy ERC20

In [ ]:
# Deploy the MyToken ERC20 with an initial supply of 100000 tokens (18 decimals)
contract_erc20_mytoken = get_contract_instance_from_bytecode(path_folder_build_contracts, "MyToken")
tx_hash = contract_erc20_mytoken.constructor(**{"initialSupply": w3.toWei(100000, unit="ether")}).transact() 
tx_receipt = w3.eth.wait_for_transaction_receipt(tx_hash)
address_erc20_mytoken = tx_receipt.contractAddress

### Deploy the AMM exchange contract `ETH/ERC20`

In [ ]:
# When deploying the MVAMM, we need to provide the ERC20 token address
contract_mvamm = get_contract_instance_from_bytecode(path_folder_build_contracts, 'MVAMM')
tx_hash = contract_mvamm.constructor(**{"_tokenAddress": address_erc20_mytoken}).transact()
tx_receipt = w3.eth.wait_for_transaction_receipt(tx_hash)
address_mvamm = tx_receipt.contractAddress 

### Instantiate MVAMM 

In [ ]:
amm_contract = {"path_json": path_to_build_folder, "name": "MVAMM", "address": address_mvamm}
erc20_contract = {"path_json": path_to_build_folder, "name": "MyToken", "address": address_erc20_mytoken}
mvamm = MVAMM(amm_contract, erc20_contract)

### Initiate liquidity

In [ ]:
mvamm.get_LP_info()

In [ ]:
mvamm.initiate_liquidity(10, 150)

In [ ]:
mvamm.get_LP_info()

### Add liquidity

In [ ]:
mvamm.add_liquidity(10, 50)

In [ ]:
mvamm.get_LP_info()

In [ ]:
mvamm.get_balance_liquidity_tokens(w3.eth.default_account)

### Remove liquidity

In [ ]:
mvamm.remove_liquidity(10)

In [ ]:
mvamm.get_LP_info()

In [ ]:
mvamm.get_balance_liquidity_tokens(w3.eth.default_account)

### Get input and output prices

In [ ]:
# check duality, see theorem 10. 
input_price_to_test = 1
input_price = float(mvamm.get_input_price(input_price_to_test))
output_price = float(mvamm.get_output_price(input_price))

input_price_to_test - 0.1 < output_price < input_price_to_test + 0.1

### Swapping eth to tokens & tokens to eth


In [ ]:
mvamm.get_LP_info(), get_balances(w3.eth.default_account, deployed_contract_erc20_mytoken)

In [ ]:
mvamm.get_input_price(20)

In [ ]:
mvamm.eth_to_token(10)

In [ ]:
mvamm.get_LP_info(), get_balances(w3.eth.default_account, deployed_contract_erc20_mytoken)

In [ ]:
mvamm.get_LP_info(), get_balances(w3.eth.default_account, deployed_contract_erc20_mytoken)

In [ ]:
mvamm.get_output_price(2)

In [ ]:
mvamm.token_to_eth(2)

In [ ]:
mvamm.get_LP_info(), get_balances(w3.eth.default_account, deployed_contract_erc20_mytoken)